In [1]:
import json
import random

from dotenv import load_dotenv
from google import genai
from google.adk.agents import Agent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search
from google.genai import types
from pydantic import BaseModel, Field

In [2]:
load_dotenv()

retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1, # Initial delay before first retry (in seconds)
    http_status_codes=[429, 500, 503, 504] # Retry on these HTTP errors
)

In [3]:
class FootballClubList(BaseModel):
    clubs: list[str] = Field(
        description="A list of exactly 10 major European football club names."
    )

In [4]:
client = genai.Client()

agent_sportscaster = Agent(
    name="sportscaster",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config,
    ),
    instruction="You are a sportscaster.",
)
runner_sportscaster = InMemoryRunner(agent=agent_sportscaster)

agent_lister = Agent(
    name="lister",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config,
    ),
    instruction="List 10 major European football clubs.",
    output_schema=FootballClubList,
)
runner_lister = InMemoryRunner(agent=agent_lister)


In [5]:
response_lister_raw = await runner_lister.run_debug(
    "List 10 major European football clubs"
)
response_lister = FootballClubList.model_validate_json(response_lister_raw[0].content.parts[0].text)
clubs =response_lister.clubs
print("clubs:", clubs)


 ### Created new session: debug_session_id

User > List 10 major European football clubs
lister > {
  "clubs": [
    "Real Madrid",
    "FC Barcelona",
    "Manchester United",
    "Liverpool FC",
    "Bayern Munich",
    "Juventus FC",
    "AC Milan",
    "Inter Milan",
    "Paris Saint-Germain",
    "Borussia Dortmund"
  ]
}
clubs: ['Real Madrid', 'FC Barcelona', 'Manchester United', 'Liverpool FC', 'Bayern Munich', 'Juventus FC', 'AC Milan', 'Inter Milan', 'Paris Saint-Germain', 'Borussia Dortmund']


In [6]:
print("Select a club:")
for i, club in enumerate(clubs, 1):
    print(f"{i}. {club}")

selection = int(input("Enter number: "))
human_club = clubs[selection - 1]
print(f"You selected: {human_club}")

Select a club:
1. Real Madrid
2. FC Barcelona
3. Manchester United
4. Liverpool FC
5. Bayern Munich
6. Juventus FC
7. AC Milan
8. Inter Milan
9. Paris Saint-Germain
10. Borussia Dortmund
You selected: Liverpool FC


In [7]:
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=f"What's the arch rival of {human_club}? Just give me the name of the club.",
)
computer_club = response.text
print(f"The arch rival of {human_club} is {computer_club}.")

The arch rival of Liverpool FC is Manchester United.


In [8]:
agent_computer = Agent(
    name="computer",
    model=Gemini(
        model="gemini-2.5-pro",
        retry_options=retry_config,
    ),
    instruction=f"""
You are the manager of the football club {computer_club}, and you are responsible for the draft. Your goal is to assemble the strongest team possible.

Draft Structure: You will draft exactly 18 players across 3 rounds, with each round offering a dynamically generated pool of 16 players.

Player Attributes: Each player has one or more natural positions (Forward, Midfielder, Defender, Goalkeeper) and a base star rating (1–6).

Opponent: H (human) is your opponent. Draft picks alternate, and you cannot select a player who has already been chosen.

Team Requirement: Your final roster must include at least 1 Goalkeeper (GK) in the starting lineup.

Star Rating Calculations (Effective Rating)

The player's effective star rating changes based on the position they are assigned:

Natural Position: Full base star rating.

Out-of-Position (Non-GK): Base stars minus 1 (e.g., a Midfielder playing Defender).

Non-Goalkeeper (Non-GK) playing as GK: Automatically reduced to 1 star.

Goalkeeper (GK) playing as a Non-GK: Automatically reduced to 1 star.
    """,
)
runner_computer = InMemoryRunner(agent=agent_computer)

agent_administrator = Agent(
    name="administrator",
    model=Gemini(
        # model="gemini-2.5-flash-lite",
        model="gemini-2.5-pro",
        retry_options=retry_config,
    ),
    instruction=f"""
        You are the administrator of the draft and the match between {human_club} and {computer_club}.
        You have the capability to create pools of real players with varying skill levels.
        The star ratings of the players should be distributed in a pyramid structure across the pool, with top-rated players at the top.
        These players can be from any team in the world. They can be from the past or present.
        Once a player is shown, he must not be shown again.
        Player positions and stars:
            * Natural positions: Forward, Midfielder, Defender, Goalkeeper
            * Stars: 1–6
            * A play can have one or multiple natural positions
    """,
)
runner_administrator = InMemoryRunner(agent=agent_administrator)

In [9]:
response_sportscaster_raw = await runner_sportscaster.run_debug(
    f"There is a draft and a match between {human_club} and {computer_club}. Briefly introduce the history of both clubs."
)


 ### Created new session: debug_session_id

User > There is a draft and a match between Liverpool FC and Manchester United. Briefly introduce the history of both clubs.
sportscaster > Alright, football fanatics, settle in, because we're about to dive into a matchup steeped in history, passion, and a rivalry that sends shivers down the spine! Tonight, we have a clash between two titans of English football: Liverpool FC and Manchester United!

Let's start with the Reds of **Liverpool FC**. Founded way back in 1892, this Merseyside club is synonymous with European glory. They've lifted that coveted European Cup/Champions League trophy a staggering **six times**, more than any other English club! Domestically, their trophy cabinet is overflowing with **19 league titles**, a record they share with their rivals, and they've also added **eight League Cups** and **seven FA Cups** to their illustrious history. Under legendary managers like Bill Shankly and Bob Paisley, Liverpool built an empir

In [10]:
human_squad = []
computer_squad = []

In [11]:
def human_choose_player(players):
    while True:
        player_choice = input("Choose a player (or stop to finish this round): ").strip()
        if player_choice in players:
            player_info = players[player_choice]
            print(f"You selected: {player_choice}")
            print(f"Player info: {player_info}")
            players.pop(player_choice)
            return {player_choice: player_info}
        elif player_choice == "stop":
            return None
        else:
            print("Invalid player. Please try again.")

def computer_choose_player(runner_computer, players):
    player_choice = random.choice(list(players.keys()))
    print(f"Computer selected: {player_choice}")
    player_info = players[player_choice]
    print(f"Player info: {player_info}")
    players.pop(player_choice)
    return {player_choice: player_info}


In [12]:
def print_team_formation(players):
    """
    Prints players grouped by position in the order: 
    Forward, Midfielder, Defender, Goalkeeper.
    
    Args:
        players: List of dicts or objects with 'name', 'position', and 'stars'.
    """
    # 1. Define the explicit print order
    order = ["Forward", "Midfielder", "Defender", "Goalkeeper"]
    
    # 2. Group players by position
    # Use a dictionary to hold lists for each position
    squad = {pos: [] for pos in order}

    for player in players:
        for name, info in player.items():
            positions = info['positions']
            for pos in positions:
                if pos in squad:
                    squad[pos].append([name, info])
                    break
    
    # 3. Print strictly in the defined order
    for pos in order:
        if squad[pos]:  # Only print if there are players in this position
            print(f"{pos}s")
            print("-" * 20)
            for each in squad[pos]:
                print(each)
            print()

In [13]:
async def draft_players():
    response_administrator_raw = await runner_administrator.run_debug(
        f"""
        Create a pool of 16 real players with varying skill levels from 1 star to 6 stars.
        These players will be used for the draft.
        Although these players may play on a team in reality, for this draft they are all free agents and available to be selected.
        Each player's name, natural position(s), and star rating should be shown.
        The star ratings should be distributed in a pyramid structure across the pool, with top-rated players at the top.

        The output must be a dictionary, whose keys are the player names and values are dictionaries with the keys "positions" and "stars".
        The output mustn't include "```json" or "```". 
        """
    )
    response_administrator = response_administrator_raw[0].content.parts[0].text.replace("```json", "").replace("```", "")
    players = json.loads(response_administrator)
    for name, info in players.items():
        print(name, info)

    human_finished = False
    computer_finished = False

    while players and (not human_finished or not computer_finished):
        if len(human_squad) >= 18:
            human_finished = True
        if len(computer_squad) >= 18:
            computer_finished = True

        print()
        print("players to choose from:")
        for name, info in players.items():
            print(name, info)
        print()

        if not human_finished:
            human_choice = human_choose_player(players)
            if human_choice:
                human_squad.append(human_choice)
            else:
                human_finished = True

        if not computer_finished:
            computer_choice = computer_choose_player(runner_computer, players)
            if computer_choice:
                computer_squad.append(computer_choice)
            else:
                computer_finished = True

        print("human_squad size:", len(human_squad))
        print("computer_squad size:", len(computer_squad))
        print()

        print("human_squad:")
        print_team_formation(human_squad)
        print("computer_squad:")
        print_team_formation(computer_squad)
        print()


In [14]:
for i in range(3):
    await draft_players()


 ### Created new session: debug_session_id

User > 
        Create a pool of 16 real players with varying skill levels from 1 star to 6 stars.
        These players will be used for the draft.
        Although these players may play on a team in reality, for this draft they are all free agents and available to be selected.
        Each player's name, natural position(s), and star rating should be shown.
        The star ratings should be distributed in a pyramid structure across the pool, with top-rated players at the top.

        The output must be a dictionary, whose keys are the player names and values are dictionaries with the keys "positions" and "stars".
        The output mustn't include "```json" or "```". 
        
administrator > {
    "Lionel Messi": {
        "positions": ["Forward", "Midfielder"],
        "stars": 6
    },
    "Kevin De Bruyne": {
        "positions": ["Midfielder"],
        "stars": 5
    },
    "Virgil van Dijk": {
        "positions": ["Defender"],
  

In [15]:
human_squad

[{'Lionel Messi': {'positions': ['Forward', 'Midfielder'], 'stars': 6}},
 {'Virgil van Dijk': {'positions': ['Defender'], 'stars': 5}},
 {'Alisson Becker': {'positions': ['Goalkeeper'], 'stars': 4}},
 {'Harry Kane': {'positions': ['Forward'], 'stars': 4}},
 {'Luka Modrić': {'positions': ['Midfielder'], 'stars': 4}},
 {'Paolo Maldini': {'positions': ['Defender'], 'stars': 4}},
 {'Cristiano Ronaldo': {'positions': ['Forward'], 'stars': 6}},
 {'Erling Haaland': {'positions': ['Forward'], 'stars': 5}},
 {'Mohamed Salah': {'positions': ['Forward'], 'stars': 5}},
 {'Casemiro': {'positions': ['Midfielder'], 'stars': 4}},
 {'Thibaut Courtois': {'positions': ['Goalkeeper'], 'stars': 4}},
 {'Kylian Mbappé': {'positions': ['Forward'], 'stars': 6}},
 {'Jude Bellingham': {'positions': ['Midfielder'], 'stars': 5}},
 {'Rúben Dias': {'positions': ['Defender'], 'stars': 5}},
 {'Heung-min Son': {'positions': ['Forward'], 'stars': 4}},
 {'Ronaldinho': {'positions': ['Midfielder', 'Forward'], 'stars': 4}}

In [16]:
computer_squad

[{'Kevin De Bruyne': {'positions': ['Midfielder'], 'stars': 5}},
 {'Diogo Jota': {'positions': ['Forward'], 'stars': 3}},
 {'Scott McTominay': {'positions': ['Midfielder'], 'stars': 2}},
 {'John Stones': {'positions': ['Defender', 'Midfielder'], 'stars': 3}},
 {'Kyle Walker': {'positions': ['Defender'], 'stars': 3}},
 {'Alexis Mac Allister': {'positions': ['Midfielder'], 'stars': 3}},
 {'Wout Weghorst': {'positions': ['Forward'], 'stars': 2}},
 {'Yann Sommer': {'positions': ['Goalkeeper'], 'stars': 3}},
 {'Ben Doak': {'positions': ['Forward'], 'stars': 1}},
 {'Aaron Wan-Bissaka': {'positions': ['Defender'], 'stars': 2}},
 {'Dominik Szoboszlai': {'positions': ['Midfielder'], 'stars': 3}},
 {'Trent Alexander-Arnold': {'positions': ['Defender'], 'stars': 4}},
 {'Andrew Robertson': {'positions': ['Defender'], 'stars': 3}},
 {'Zinedine Zidane': {'positions': ['Midfielder'], 'stars': 4}},
 {'Lisandro Martínez': {'positions': ['Defender'], 'stars': 3}},
 {'Christian Eriksen': {'positions': ['

In [17]:
def get_squad_stars(squad):
    stars = 0
    # player is a dict: the key is the player name, the value is a dict with keys "positions" and "stars"
    for player in squad:
        for name, info in player.items():
            stars += info['stars']
    return stars

human_squad_stars = get_squad_stars(human_squad)
computer_squad_stars = get_squad_stars(computer_squad)

print("human_squad_stars:", human_squad_stars)
print("computer_squad_stars:", computer_squad_stars)

human_squad_stars: 83
computer_squad_stars: 50
